**Facultad de Ciencias Exactas, Ingeniería y Agrimensura - UNR**

Tecnicatura Universitaria en Inteligencia Artificial

# Procesamiento del Lenguaje Natural - Trabajo Práctico N°: 1.

Integrantes:
- Pace, Bruno. Legajo: P-5295/7.
- Sancho Almenar, Mariano. Legajo: S-5778/9.

In [1]:
# ==========================
# Instalación de Paquetes Externos
# ==========================
!pip install sentiment_analysis_spanish
!pip install transformers
!pip install https://github.com/explosion/spacy-models/releases/download/es_core_news_lg-3.5.0/es_core_news_lg-3.5.0.tar.gz
!pip install gliner
!python -m spacy download en_core_web_sm

# ==========================
# Importación de Librerías
# ==========================
# Librerías básicas para manejo de datos
import pandas as pd
import warnings

# NLP y procesamiento de texto
import spacy
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Modelos y herramientas de ML
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Análisis de sentimientos en español
from sentiment_analysis_spanish import sentiment_analysis

# Librerías de transformers
from gliner import GLiNER
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import MarianMTModel, MarianTokenizer
from transformers import pipeline

# Configuración de warnings
warnings.filterwarnings('ignore')

# Descarga de recursos adicionales de NLTK
nltk.download('punkt')
nltk.download('stopwords')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.0/30.0 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.0/568.0 MB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.6/152.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 919.6/919.6 kB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.8 MB/s eta 0:00:00
  Created wheel for es_core_news_lg: filename=es_core_news_lg-3.5.0-py3-none-any.whl size=567975316 sha256=864fc8d5dd8caddcec053a19f0f2974cc7f3d21f75e33079befe7653cc986fc6
  Stored in directory: /root/.cache/pip/wheels/6c/1c/88/21043ad68f0560b1ddae

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## Prompts

Ingreso del usuario mediante teclado de los prompts a evaluar por el programa.

In [2]:
def sentiment_input() -> str:
  """
  Inicia el programa y pide al usuario que ingrese un prompt.
  """
  print('¡Bienvenido al recomendador de actividades!')
  sentiment_input = input("¿Cómo te sientes hoy?: ")
  return sentiment_input

In [3]:
def preference_input() -> str:
  """
  Pide al usuario que ingrese su temática a explorar.
  """
  preference_input = input("¿Qué tipo de temática querés abordar?: ")
  return preference_input

## Modelado
- Se utiliza transformers para clasificar el estado de ánimo de la persona. Este puede ser: "Alegre", "Melancólico" o "Ni fu ni fa".

### Transformers

In [4]:
# Cargamos el tokenizador y el modelo.
model_name: str = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer: BertTokenizer = BertTokenizer.from_pretrained(model_name)
model: BertForSequenceClassification = BertForSequenceClassification.from_pretrained(model_name)


# Creación de pipeline.
nlp: pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [5]:
def get_sentiment(text: str) -> str:
    """
    Retorna el sentimiento de un texto, mediante el uso de transformers.
    """
    label: str = nlp(text)[0]['label']
    score: float = nlp(text)[0]['score']

    if label in ['5 stars', '4 stars']:
        print(score)
        return 'Alegre'
    elif label == '3 stars':
        print(score)
        return 'Ni fu ni fa'
    elif label in ['2 stars', '1 star']:
        print(score)
        return 'Melancólico'

In [40]:
get_sentiment("i'm relaxed")

0.41296347975730896


'Alegre'

In [6]:
test_list: list[str] = [
    "Estoy muy feliz",
    "Me siento entusiasmado",
    "Estoy tranquilo",
    "Me siento relajado",
    "Estoy aburrido",
    "Me siento nostálgico",
    "Estoy un poco triste",
    "Me siento melancólico",
    "Estoy frustrado",
    "Me siento ansioso",
    "Estoy enojado",
    "Me siento decepcionado",
    "Estoy muy emocionado",
    "Me siento inspirado",
    "Estoy cansado",
    "Me siento agotado",
    "Estoy preocupado",
    "Me siento optimista",
    "Estoy motivado",
    "Me siento agradecido",
    "Estoy desconcentrado",
    "Me siento esperanzado",
    "Estoy estresado",
    "Me siento insatisfecho",
    "Estoy satisfecho",
    "Me siento apático",
    "Estoy en paz",
    "Me siento irritado",
    "Estoy un poco nervioso",
    "Me siento confiado",
    "Estoy renovado",
    "Me siento pleno",
    "Estoy resignado",
    "Me siento inquieto",
    "Estoy contento",
    "Me siento vulnerable",
    "Estoy enamorado",
    "Me siento frustrado",
    "Estoy ilusionado",
    "Me siento reflexivo",
    "Estoy agradecido",
    "Me siento emocionalmente agotado"
]

for text in test_list:
  print(f'{text} -> {get_sentiment(text)}')

0.7204190492630005
Estoy muy feliz -> Alegre
0.5104050636291504
Me siento entusiasmado -> Alegre
0.4654117822647095
Estoy tranquilo -> Alegre
0.3073946535587311
Me siento relajado -> Melancólico
0.6528457999229431
Estoy aburrido -> Melancólico
0.32323989272117615
Me siento nostálgico -> Alegre
0.5161958932876587
Estoy un poco triste -> Ni fu ni fa
0.3490581214427948
Me siento melancólico -> Melancólico
0.6465574502944946
Estoy frustrado -> Melancólico
0.45507076382637024
Me siento ansioso -> Alegre
0.6622191667556763
Estoy enojado -> Melancólico
0.469738245010376
Me siento decepcionado -> Melancólico
0.6982787847518921
Estoy muy emocionado -> Alegre
0.38369399309158325
Me siento inspirado -> Alegre
0.6665818095207214
Estoy cansado -> Melancólico
0.40258100628852844
Me siento agotado -> Melancólico
0.35649794340133667
Estoy preocupado -> Melancólico
0.30568787455558777
Me siento optimista -> Ni fu ni fa
0.4021136164665222
Estoy motivado -> Alegre
0.45900627970695496
Me siento agradecido

### sentiment_analysis_spanish library

A fin de realizar una comparación, utilizamos sentiment-analysis sobre la misma lista de test.

El rendimiento obtenido es inferior, teniendo en cuenta que un resultado cercano a uno es un texto positivo, cercano a cero es negativo y en los valores medios es neutro. Por eso se optó por los transformers.

Para obtener esta conclusión se visitó la [Documentación](https://pypi.org/project/sentiment-analysis-spanish/).

In [7]:
sentiment = sentiment_analysis.SentimentAnalysisSpanish()

In [8]:
for texto in test_list:
  print(f'{texto} -> {sentiment.sentiment(texto)}')

Estoy muy feliz -> 0.6889487711193526
Me siento entusiasmado -> 0.0728752984896165
Estoy tranquilo -> 0.7826378949098491
Me siento relajado -> 0.23951579470116582
Estoy aburrido -> 0.31062133515766494
Me siento nostálgico -> 0.0728752984896165
Estoy un poco triste -> 0.04078651378014973
Me siento melancólico -> 0.0728752984896165
Estoy frustrado -> 0.31062133515766494
Me siento ansioso -> 0.0728752984896165
Estoy enojado -> 0.31062133515766494
Me siento decepcionado -> 0.005644903897282337
Estoy muy emocionado -> 0.537734459784737
Me siento inspirado -> 0.0728752984896165
Estoy cansado -> 0.31062133515766494
Me siento agotado -> 0.0728752984896165
Estoy preocupado -> 0.31062133515766494
Me siento optimista -> 0.0728752984896165
Estoy motivado -> 0.31062133515766494
Me siento agradecido -> 0.0728752984896165
Estoy desconcentrado -> 0.31062133515766494
Me siento esperanzado -> 0.0728752984896165
Estoy estresado -> 0.31062133515766494
Me siento insatisfecho -> 0.0728752984896165
Estoy sat

### NER

Se emplea para la identificación de entidades que facilitarán la búsqueda en los datos, entendiendo y analizando tanto sus preferencias como el contexto.

In [9]:
ner = spacy.load('es_core_news_lg')

In [10]:
for texto in test_list:
  doc = ner(texto)
  for ent in doc.ents:
    print(f'Entidad: {ent.text}, Etiqueta: {ent.label_}, Explicación: {spacy.explain(ent.label_)}')

Entidad: Estoy muy feliz, Etiqueta: MISC, Explicación: Miscellaneous entities, e.g. events, nationalities, products or works of art
Entidad: Estoy enojado, Etiqueta: MISC, Explicación: Miscellaneous entities, e.g. events, nationalities, products or works of art
Entidad: Estoy cansado, Etiqueta: MISC, Explicación: Miscellaneous entities, e.g. events, nationalities, products or works of art
Entidad: Estoy estresado, Etiqueta: MISC, Explicación: Miscellaneous entities, e.g. events, nationalities, products or works of art
Entidad: Estoy en paz, Etiqueta: MISC, Explicación: Miscellaneous entities, e.g. events, nationalities, products or works of art
Entidad: Estoy enamorado, Etiqueta: MISC, Explicación: Miscellaneous entities, e.g. events, nationalities, products or works of art


In [11]:
!pip install gliner
from gliner import GLiNER

gliner_model = GLiNER.from_pretrained("urchade/gliner_multi-v2.1")

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

gliner_config.json:   0%|          | 0.00/477 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.77k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.16G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

In [12]:
gliner_model.eval()

labels: list[str] = ["person", "book", "location", "date", "actor", "character","game"]


for text in test_list:
  entities = gliner_model.predict_entities(text, labels, threshold=0.4)
  for entity in entities:
    print(entity, "=>", entity["label"])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


KeyboardInterrupt: 

## Description processing

In [17]:
df_libros: pd.DataFrame = pd.read_csv('books.csv')
df_peliculas: pd.DataFrame = pd.read_csv('IMDB-Movie-Data.csv')
df_juegos: pd.DataFrame = pd.read_csv('bgg_database.csv')

In [20]:
df_libros.columns

Index(['Unnamed: 0', 'title', 'link', 'summary'], dtype='object')

In [ ]:
df_libros_union = df_librs.drop(columns=[])

In [18]:
df_total: pd.DataFrame = pd.concat([df_libros, df_peliculas, df_juegos], axis=0)

In [19]:
df_total.sample(5)

,Unnamed: 0,title,link,summary,Title,Description,game_name,description
819,NaN,NaN,NaN,NaN,Wolves at the Door,Four friends gather at an elegant home during ...,NaN,NaN
963,963.0,On the Fourfold Root of the Principle of Suffi...,/ebooks/50966,"""On the Fourfold Root of the Principle of Suff...",NaN,NaN,NaN,NaN
257,NaN,NaN,NaN,NaN,NaN,NaN,MicroMacro: Crime City,"Crimes have taken place all over the city, and..."
923,NaN,NaN,NaN,NaN,No Escape,"In their new overseas home, an American family...",NaN,NaN
360,NaN,NaN,NaN,NaN,Step Brothers,Two aimless middle-aged losers still living at...,NaN,NaN


Normalización del texto, conversion a minuscula

In [ ]:
df_peliculas['description'] = df_peliculas['Description'].str.lower()
df_libros['summary'] = df_libros['summary'].str.lower()
df_juegos['Description'] = df_juegos['description'].str.lower()
df_peliculas['Title'] = df_peliculas['Title'].str.lower()
df_libros['title'] = df_libros['title'].str.lower()
df_juegos['game_name'] = df_juegos['game_name'].str.lower()

Remoción de stopwords con nltk

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

stop_words = set(stopwords.words('english'))

def remove_stopwords(text: str) -> str:
    """
    Remueve stopwords de un texto en ingles. Retorna el texto sin stopwords.
    """
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word.casefold() not in stop_words]
    return " ".join(filtered_text)

In [ ]:
df_libros.info()

In [ ]:
df_libros = df_libros.dropna()

In [ ]:
df_peliculas['description_no_stopwords'] = df_peliculas['description'].apply(remove_stopwords)
df_libros['summary_no_stopwords'] = df_libros['summary'].apply(remove_stopwords)
df_juegos['Description_no_stopwords'] = df_juegos['Description'].apply(remove_stopwords)

Lematización del texto

In [ ]:
lemmatizer = spacy.load("en_core_web_sm")

def lematizar_texto(texto: str) -> str:
    """
    Función para lematizar un texto en inglés
    """
    lemmatizer_ = lemmatizer(texto)
    lemmas = [tok.lemma_.lower() for tok in lemmatizer_]
    return ' '.join(lemmas)

In [ ]:
df_libros['summary_lematizado'] = df_libros['summary_no_stopwords'].apply(lematizar_texto)
df_peliculas['description_lematizado'] = df_peliculas['description_no_stopwords'].apply(lematizar_texto)
df_juegos['Description_lematizado'] = df_juegos['Description_no_stopwords'].apply(lematizar_texto)

In [ ]:
df_libros.sample(8)

## modelo

In [ ]:
# Calcular TF-IDF: ingeniería de características de peliculas
tfidf_vectorizer_peliculas = TfidfVectorizer()

tfidf_matrix_peliculas = tfidf_vectorizer_peliculas.fit_transform(df_peliculas['description_lematizado'])

In [ ]:
# Calcular TF-IDF: ingeniería de características de libros
tfidf_vectorizer_libros = TfidfVectorizer()

tfidf_matrix_libros = tfidf_vectorizer_libros.fit_transform(df_libros['summary_lematizado'])

In [ ]:
# Calcular TF-IDF: ingeniería de características de libros
tfidf_vectorizer_juegos = TfidfVectorizer()

tfidf_matrix_juegos = tfidf_vectorizer_juegos.fit_transform(df_juegos['Description_lematizado'])

print(tfidf_matrix_juegos.shape)
tfidf_matrix_libros

In [ ]:
df_juegos['Description_lematizado']

In [ ]:
# Define el modelo y el tokenizador
modelo = 'Helsinki-NLP/opus-mt-es-en'
tokenizer = MarianTokenizer.from_pretrained(modelo)
model = MarianMTModel.from_pretrained(modelo)

In [ ]:
def traducir_frase_usuario(frase_usuario: str) -> str:
    """
    Traduce una frase de español a inglés.
    """
    # Tokeniza el texto y genera la traducción
    inputs = tokenizer(frase_usuario, return_tensors="pt")
    outputs = model.generate(**inputs)

    # Decodifica y muestra la traducción
    texto_ingles = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return texto_ingles

In [ ]:
traducir_frase_usuario("Quisiera ver una pelicula feliz")

In [ ]:
def procesar_frase_usuario(text: str) -> str:
    """
    Función para procesar una frase ingresada por el usuario.
    """
    text = traducir_frase_usuario(text)
    text = text.lower()
    text = remove_stopwords(text)
    text = lematizar_texto(text)
    return text

In [ ]:
def obtener_similitud_usuario_peliculas(frase_usuario: str) -> pd.DataFrame:
    """
    Calcula la similitud de coseno entre una frase ingresada por el usuario
    y las descripciones de las películas usando la matriz TF-IDF.
    """
    # Procesar la frase del usuario
    frase_usuario = procesar_frase_usuario(frase_usuario)
    # Transformar la frase del usuario en la misma representación TF-IDF
    tfidf_usuario = tfidf_vectorizer_peliculas.transform([frase_usuario])

    # Calcular la similitud de coseno entre la frase del usuario y la matriz TF-IDF de las películas
    similitudes = cosine_similarity(tfidf_usuario, tfidf_matrix_peliculas)

    # Crear un DataFrame con los resultados, para ver las similitudes
    df_similitudes = pd.DataFrame(similitudes.T, columns=['similitud'], index=df_peliculas.index)

    # Agregar las películas a la tabla
    df_similitudes['titulo'] = df_peliculas['Title']

    # Ordenar por similitud de mayor a menor
    df_similitudes = df_similitudes.sort_values(by='similitud', ascending=False)

    return df_similitudes


In [ ]:
def obtener_similitud_usuario_juegos(frase_usuario: str) -> pd.DataFrame:
    """
    Calcula la similitud de coseno entre una frase ingresada por el usuario
    y las descripciones de los juegos usando la matriz TF-IDF.
    """
    # Procesar la frase del usuario
    frase_usuario = procesar_frase_usuario(frase_usuario)

    # Transformar la frase del usuario en la misma representación TF-IDF
    tfidf_usuario = tfidf_vectorizer_juegos.transform([frase_usuario])

    # Calcular la similitud de coseno entre la frase del usuario y la matriz TF-IDF de los juegos
    similitudes = cosine_similarity(tfidf_usuario, tfidf_matrix_juegos)

    # Crear un DataFrame con los resultados, para ver las similitudes
    df_similitudes = pd.DataFrame(similitudes.T, columns=['similitud'], index=df_juegos.index)

    # Agregar los títulos de los juegos a la tabla
    df_similitudes['titulo'] = df_juegos['game_name']

    # Ordenar por similitud de mayor a menor
    df_similitudes = df_similitudes.sort_values(by='similitud', ascending=False)

    return df_similitudes


In [ ]:
obtener_similitud_usuario_juegos("Quisiera ver algo de ciencia ficción")

In [ ]:
def obtener_similitud_usuario_libros(frase_usuario: str) -> pd.DataFrame:
    """
    Calcula la similitud de coseno entre una frase ingresada por el usuario
    y las descripciones de los libros usando la matriz TF-IDF.
    """
    # Procesar la frase del usuario
    frase_usuario = procesar_frase_usuario(frase_usuario)  # Asegúrate de tener esta función definida

    # Transformar la frase del usuario en la misma representación TF-IDF
    tfidf_usuario = tfidf_vectorizer_libros.transform([frase_usuario])

    # Calcular la similitud de coseno entre la frase del usuario y la matriz TF-IDF de los libros
    similitudes = cosine_similarity(tfidf_usuario, tfidf_matrix_libros)

    # Crear un DataFrame con los resultados de similitud
    df_similitudes = pd.DataFrame(similitudes.T, columns=['similitud'], index=df_libros.index)

    # Agregar los títulos de los libros al DataFrame
    df_similitudes['titulo'] = df_libros['Title']

    # Ordenar los resultados por similitud de mayor a menor
    df_similitudes = df_similitudes.sort_values(by='similitud', ascending=False)

    return df_similitudes


In [ ]:
obtener_similitud_usuario_libros("Quisiera ver un libro de ciencia ficción")

In [ ]:
def get_prompt_recommendation(sentiment: str, preference: str) -> list[str]:
  """
  Recibe el sentimiento y el prompt de preferencia, devuelve una lista de strings con las recomendaciones.
  """
  pass

In [ ]:
def get_prompt() -> str:
  """
  Pide al usuario que ingrese un prompt.
  """
  prompt = input("Ingrese una preferencia de actividad: ")
  return prompt

In [25]:
!pip install sentence-transformers

from sentence_transformers import SentenceTransformer, util
modelo = SentenceTransformer('msmarco-MiniLM-L-6-v3')

In [29]:
# Generar incrustaciones para todas las consultas y respuestas
incrustaciones_consultas = modelo.encode("I'd like to watch a horror film")
incrustaciones_respuestas = modelo.encode(df_peliculas['Description'])

In [30]:
df_peliculas.columns

Index(['Title', 'Description'], dtype='object')

In [35]:
#pasarle todo el data, transformar la columna summary a un description

# Encontrar la respuesta con la mejor similitud para cada consulta
#for i, incrustacion_consulta in enumerate(incrustaciones_consultas):
similitudes = util.cos_sim(incrustaciones_consultas, incrustaciones_respuestas)[0]
#mejor_indice = similitudes.argmax()
#print(f"Consulta: '¿")
mejor_indice = similitudes.argmax().item()  # Convertir a entero
print(f"Mejor respuesta (Similitud: {similitudes[mejor_indice]:.4f}): {df_peliculas['Description'].iloc[mejor_indice]}")

print()


Mejor respuesta (Similitud: 0.3947): A documentary team films the lives of a group of vampires for a few months. The vampires share a house in Wellington, New Zealand. Turns out vampires have their own domestic problems too.



- juntar datasets.
- crear columna o estructura que le pase description a estado de animo para machear.
- que machee con el estado de ánimo en primer lugar, luego con el libro, juego o pelicula que le siga.
- acomodar los data en el archivo de datasets.
- informe.